## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

df = spark.read.csv(file_location,header=True, inferSchema=True)

In [0]:
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
## Handling categorical features
from pyspark.ml.feature import StringIndexer


In [0]:
indexer=StringIndexer(inputCols=["sex","smoker","day","time"],outputCols=["sex_indexed","smoker_indexed","day_indexed","time_indexed"])
df_r = indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|           0.0|        1.0|         0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|


In [0]:
from pyspark.ml.feature import VectorAssembler
featureAssembler=VectorAssembler(inputCols=["tip","sex_indexed","size","smoker_indexed","day_indexed","time_indexed"],outputCol="Independent Feature")
output = featureAssembler.transform(df_r)

In [0]:
output.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed| Independent Feature|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|           0.0|        1.0|         0.0|[1.01,1.0,2.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[1.66,0.0,3.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|           0.0|        1.0|         0.0|[3.5,0.0,3.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|           0.0|        1.0|         0.0|[3.31,0.0,2.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|           0.0|        1.0|         0.0|[3.61,1.0,4.0,0.0...|
|     25.29|4.71|  Male|    No|S

In [0]:
output.select('Independent Feature').show()

+--------------------+
| Independent Feature|
+--------------------+
|[1.01,1.0,2.0,0.0...|
|[1.66,0.0,3.0,0.0...|
|[3.5,0.0,3.0,0.0,...|
|[3.31,0.0,2.0,0.0...|
|[3.61,1.0,4.0,0.0...|
|[4.71,0.0,4.0,0.0...|
|[2.0,0.0,2.0,0.0,...|
|[3.12,0.0,4.0,0.0...|
|[1.96,0.0,2.0,0.0...|
|[3.23,0.0,2.0,0.0...|
|[1.71,0.0,2.0,0.0...|
|[5.0,1.0,4.0,0.0,...|
|[1.57,0.0,2.0,0.0...|
|[3.0,0.0,4.0,0.0,...|
|[3.02,1.0,2.0,0.0...|
|[3.92,0.0,2.0,0.0...|
|[1.67,1.0,3.0,0.0...|
|[3.71,0.0,3.0,0.0...|
|[3.5,1.0,3.0,0.0,...|
|(6,[0,2],[3.35,3.0])|
+--------------------+
only showing top 20 rows



In [0]:
final_data =output.select('Independent Feature','total_bill')

In [0]:
final_data.show()

+--------------------+----------+
| Independent Feature|total_bill|
+--------------------+----------+
|[1.01,1.0,2.0,0.0...|     16.99|
|[1.66,0.0,3.0,0.0...|     10.34|
|[3.5,0.0,3.0,0.0,...|     21.01|
|[3.31,0.0,2.0,0.0...|     23.68|
|[3.61,1.0,4.0,0.0...|     24.59|
|[4.71,0.0,4.0,0.0...|     25.29|
|[2.0,0.0,2.0,0.0,...|      8.77|
|[3.12,0.0,4.0,0.0...|     26.88|
|[1.96,0.0,2.0,0.0...|     15.04|
|[3.23,0.0,2.0,0.0...|     14.78|
|[1.71,0.0,2.0,0.0...|     10.27|
|[5.0,1.0,4.0,0.0,...|     35.26|
|[1.57,0.0,2.0,0.0...|     15.42|
|[3.0,0.0,4.0,0.0,...|     18.43|
|[3.02,1.0,2.0,0.0...|     14.83|
|[3.92,0.0,2.0,0.0...|     21.58|
|[1.67,1.0,3.0,0.0...|     10.33|
|[3.71,0.0,3.0,0.0...|     16.29|
|[3.5,1.0,3.0,0.0,...|     16.97|
|(6,[0,2],[3.35,3.0])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [0]:
from pyspark.ml.regression import LinearRegression
## train test split
train_data, test_data= final_data.randomSplit([0.8,0.2])
regressor= LinearRegression(featuresCol="Independent Feature", labelCol="total_bill")
regressor=regressor.fit(train_data)

In [0]:
regressor.coefficients

Out[13]: DenseVector([3.0297, -1.1, 3.8009, 1.7581, 0.0205, -1.4844])

In [0]:
regressor.intercept

Out[14]: 1.2230647337592093

In [0]:
## predictions
pred_results = regressor.evaluate(test_data)

In [0]:
#final comparison
pred_results.predictions.show()

+--------------------+----------+------------------+
| Independent Feature|total_bill|        prediction|
+--------------------+----------+------------------+
|(6,[0,2],[1.45,2.0])|      9.55|13.218016916079343|
|(6,[0,2],[1.47,2.0])|     10.77|13.278611293261793|
|(6,[0,2],[1.75,2.0])|     17.82| 14.12693257381609|
|(6,[0,2],[1.97,2.0])|     12.02|14.793470722823038|
|(6,[0,2],[2.01,2.0])|     20.23|14.914659477187937|
|(6,[0,2],[2.34,4.0])|     17.81| 23.51632653729088|
| (6,[0,2],[2.5,4.0])|     18.35|24.001081554750478|
| (6,[0,2],[4.3,2.0])|      21.7|21.852715664578444|
|[1.0,0.0,2.0,1.0,...|      12.6|13.612697745060839|
|[1.1,1.0,2.0,1.0,...|      12.9| 12.81564330407526|
|[1.44,0.0,2.0,0.0...|      7.56|11.744452206974337|
|[1.56,0.0,2.0,0.0...|      9.94|13.571828501315458|
|[1.68,1.0,2.0,0.0...|     13.42|11.371558406265908|
|[1.71,0.0,2.0,0.0...|     10.27|14.026286330183833|
|[1.76,0.0,2.0,1.0...|     11.24|15.915284077993933|
|[1.83,1.0,1.0,0.0...|     10.07| 8.0250863168

In [0]:
# preformance matrices
pred_results.r2,pred_results.meanAbsoluteError,pred_results.meanSquaredError

Out[17]: (0.37894825353131434, 4.521865271292668, 39.84493449469329)